# 1. Imports

In [22]:
import pandas as pd
import datetime

# 2. Reading the Data Into Dataframes

# 1.1 Reading functions

In [23]:
circle_tweet_content_df = pd.read_csv('circle_tweet_content.csv')
circle_tweets_df = pd.read_csv('circle_tweets.csv')
engagements_df = pd.read_csv('engagements.csv')
likes_df = pd.read_csv('likes.csv')
shows_df = pd.read_csv('shows.csv')
ad_target_df = pd.read_csv('interests.csv')
tweet_content_df = pd.read_csv('tweet_content.csv')
tweets_df = pd.read_csv('tweets.csv')

# 3. Cleaning Data

# 3.1 Cleaning the Advertisement Data

### 3.1.1 Defining Functions

In [24]:
def count_and_merge(data):
    '''
    This function takes in a dataframe and returns a dataframe with merged values.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    df = data.groupby(['targeting type', 'targeting value']).size().reset_index(name='counts')
    df.sort_values(by=['counts'], ascending=False, inplace=True)
    return df

In [25]:
def targeting_type_frequency(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of each targeting type.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    df = data.groupby(['targeting type']).size().reset_index(name='counts')
    df.sort_values(by=['counts'], ascending=False, inplace=True)
    return df

In [26]:
def targeting_value_frequency(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of each targeting value.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    df = data.groupby(['targeting value']).size().reset_index(name='counts')
    df.sort_values(by=['counts'], ascending=False, inplace=True)
    return df

In [27]:
def convert_to_date(date):
    '''
    This function takes in a date in the format of a string and returns a date in the format of a datetime object.
    ---
    Parameters:
        date: str
    ---
    Returns:
        date: datetime object
    '''
    date = datetime.datetime.strptime(date, '%Y-%m-%d')
    return date

In [28]:
def sort_by_date(df):
    '''
    This function takes in a dataframe and returns a dataframe sorted by date.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    for i, row in df.iterrows():
        date = convert_to_date(row['date'])
        df.at[i, 'date'] = date
    df = df.sort_values(by=['date'], ascending=True)
    return df

In [29]:
def remove_hashtags(data):
    '''
    This function takes in a dataframe and returns a dataframe with the hashtags removed.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''

    for i, row in data.iterrows():
        value = row['targeting value']
        if value[0] == '#':
            value = value[1:]
        if len(value) > 3 and value[0:2] == 'the':
            value = value[2:]
        data.at[i, 'targeting value'] = value
    return data

### 3.1.2 Cleaning the Advertisment Data Using the Defined Functions

In [30]:
ad_target_df = remove_hashtags(ad_target_df)
ad_target_df_cleaned = count_and_merge(data=ad_target_df) # this dataframe contains the merged values (targeting type, targeting value), frequency of the merged values
targeting_freq_df = targeting_type_frequency(data=ad_target_df) # this dataframe contains the frequency of each targeting type
value_freq_df = targeting_value_frequency(data=ad_target_df) # this dataframe contains the frequency of each targeting value even if the targeting types are different for two merged targeting values provided that the values are the same
target_date_df = sort_by_date(df=ad_target_df) # this dataframe is sorted by date (earliest to latest)

In [31]:
ad_target_df_cleaned.to_csv('ad_target_df_cleaned.csv', index=False)
targeting_freq_df.to_csv('targeting_freq_df.csv', index=False)
value_freq_df.to_csv('value_freq_df.csv', index=False)
target_date_df.to_csv('target_date_df.csv', index=False)

In [33]:
value_freq_df

,targeting value,counts
815,turkey,1099
569,ios,689
6,18 and up,662
466,english,647
643,netflix,467
...,...,...
227,@richardbranson,1
607,lmao,1
605,list,1
604,like,1


## 3.2 Cleaning the Tweet Data

### 3.2.1 Defining Functions

In [34]:
def tweet_by_hour(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of tweets by hour.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    df = data.groupby(['hour']).size().reset_index(name='counts')
    df.sort_values(by=['counts'], ascending=False, inplace=True)
    return df

In [35]:
def tweet_by_day(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of tweets by day.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    df = data.groupby(['date']).size().reset_index(name='counts')
    df.sort_values(by=['counts'], ascending=False, inplace=True)
    return df

In [36]:
def most_active_hour_per_day(data):
    '''
    This function takes in a dataframe and returns a dataframe with the most active hour per day.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''

    data['date'] = pd.to_datetime(data['date'])
    result_dfs = []

    for date, group in data.groupby(data['date'].dt.date):
        most_active_hour = group['hour'].value_counts().idxmax()
        result_dfs.append(pd.DataFrame({'date': [date], 'most active hour': [most_active_hour]}))

    result_df = pd.concat(result_dfs, ignore_index=True)
    result_df.sort_values(by=['date'], ascending=False, inplace=True)

    return result_df


In [37]:
def language_frequency(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of each language.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    df = data.groupby(['language']).size().reset_index(name='counts')
    df.sort_values(by=['counts'], ascending=False, inplace=True)
    return df

In [38]:
def likes_per_day(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of likes per day.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    # Assuming the column containing likes is named 'likes'
    df = data.groupby(['date'])['likes'].sum().reset_index(name='likes per day')
    df.sort_values(by=['likes per day'], ascending=False, inplace=True)
    return df

In [39]:
def retweets_per_day(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of likes per day.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    # Assuming the column containing likes is named 'likes'
    df = data.groupby(['date'])['retweets'].sum().reset_index(name='retweets per day')
    df.sort_values(by=['retweets per day'], ascending=False, inplace=True)
    return df

In [40]:
def most_common_language_per_day(data):
    '''
    This function takes in a dataframe and returns a dataframe with the most common language per day.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    # Ensure 'date' is a datetime object
    data['date'] = pd.to_datetime(data['date'])

    # Group by date and find the most common language
    result_list = []
    for date, group in data.groupby(data['date'].dt.date):
        most_common_language = group['language'].mode().iloc[0]
        result_list.append({'date': date, 'most common language': most_common_language})

    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(result_list)
    df.sort_values(by=['date'], ascending=False, inplace=True)

    return df

In [41]:
def likes_retweets_correlation(data):
    '''
    This function takes in a dataframe and returns a dataframe with the correlation between likes and retweets.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        correlation_df: pandas dataframe
    '''

    # Extract relevant columns
    df = data[['likes', 'retweets']]
    corr_matrix = df.corr()

    return corr_matrix

In [42]:
def group_by_date_time(data):
    '''
    This function takes in a dataframe and returns a dataframe with the frequency of tweets by date and time.
    ---
    Parameters:
        data: pandas dataframe
    ---
    Returns:
        df: pandas dataframe
    '''
    df = data.groupby(['date', 'hour']).size().reset_index(name='counts')
    df.sort_values(by=['counts'], ascending=False, inplace=True)
    return df

In [52]:
# Group by 'date' and 'tweet/retweet', then count the occurrences
tweets_versus_retweets = tweet_content_df.groupby(['date', 'tweet/retweet']).size().unstack(fill_value=0).reset_index()

# Rename columns
tweets_versus_retweets.columns = ['date', 'tweets', 'retweets']
tweets_versus_retweets['tweets'] = tweets_versus_retweets['tweets'].astype(int)
tweets_versus_retweets['retweets'] = tweets_versus_retweets['retweets'].astype(int)
tweets_versus_retweets['date'] = pd.to_datetime(tweets_versus_retweets['date'])

### 3.2.2 Cleaning the Tweet Data

In [43]:
tweets_by_hour_df = tweet_by_hour(data=tweets_df)
tweets_by_date_df = tweet_by_day(data=tweets_df)
mode_hour_df = most_active_hour_per_day(data=tweets_df)
tweets_language_df = language_frequency(data=tweets_df)
likes_per_day_df = likes_per_day(data=tweets_df)
retweets_per_day_df = retweets_per_day(data=tweets_df)
mode_language_df = most_common_language_per_day(data=tweets_df)
corr_matrix_likes_rt = likes_retweets_correlation(data=tweets_df)
tweets_by_datetime = group_by_date_time(data=tweets_df)

In [45]:
tweets_by_hour_df.to_csv('tweets_by_hour.csv', index=False)
tweets_by_date_df.to_csv('tweets_by_date.csv', index=False)
mode_hour_df.to_csv('mode_hour.csv', index=False)
tweets_language_df.to_csv('tweets_languages.csv', index=False)
likes_per_day_df.to_csv('likes_per_day.csv', index=False)
retweets_per_day_df.to_csv('retweets_per_day.csv', index=False)
mode_language_df.to_csv('mode_language.csv', index=False)
corr_matrix_likes_rt.to_csv('correlation matrix.csv', index=False)
tweets_by_datetime.to_csv('tweets by datetime.csv', index=False)

In [54]:
tweets_versus_retweets.to_csv('tweets_versus_retweets.csv', index=True)